<a href="https://colab.research.google.com/github/EsserMishelle/spam_vs_ham/blob/main/Spam_Detection_Machine_Learning_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spam Detection Machine Learning Classification

## Problem Statement
Email inboxes are inundated with unsolicited and low-value messages, making manual spam filtering inefficient and error-prone. This project applies supervised machine learning to automate the classification of email messages as spam or non-spam (ham). Because spam datasets are typically imbalanced, model evaluation emphasizes precision, recall, and F1-score rather than accuracy alone.


## 1. Data Loading and Preparaton

In [ ]:
import pandas as pd

df = pd.read_csv('spam.csv', encoding='latin-1')
# Copy the dataset for reference
df_raw = df.copy()

df.head()

Empty columns are removed from the dataset. Only the text and labels columns are retained for modeling.

In [ ]:
# Rename the columns as 'label' and 'message'
if 'v1' in df.columns and 'v2' in df.columns:
  df = df[['v1', 'v2']]
  df.columns =['label', 'message']

df.info()

There are no null values in the label or message columns.

##2.  Explore Data Analysis

In [ ]:
display((df['label'].value_counts()))
df['label'].value_counts(normalize=True)

To better understand differences between spam and non-spam messages, message length is compared across classes.  
The dataset is imbalanced (approximately 87% ham, 13% spam), so precision, recall, and F1-score are emphasized during evaluation.


In [ ]:
df['word_count'] = df['message'].apply(lambda x: len(x.split()))
df['word_count'].describe()


In [ ]:
import matplotlib.pyplot as plt

df.boxplot(column='word_count', by='label', grid = False, showfliers=False)
plt.title('Message Length by Class')
plt.suptitle('')
plt.xlabel('Message Type')
plt.ylabel('Word Count')

Spam messages tend to have longer average lengths than non-spam messages, while non-spam messages show greater variability.


## 3. Text Preprocessing and Feature Engineering
### Structure Cleaning/Label Cleaning

Spam messages are encoded as class 1 and non-spam (ham) messages as class 0 to support binary classification.


In [ ]:
# Converting labels to numberic. Spam will be class 1 and ham as class 0

df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

### Message Cleaning/Normalization





In [ ]:
import re

def clean_text(text):
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # Remove non-alphabetic characters
  text = re.sub(r'\s+',' ', text).strip() # normalize whitespace
  text = text.lower() # convert to lowercase
  return text

# Apply the clean_text function to the message column
df['clean_message'] = df['message'].apply(clean_text)

### CORPUS Building

In [ ]:
corpus = df['clean_message'].tolist()

The corpus contains the cleaned message text used for feature extraction.

### Vectorization

Text messages are converted into numerical features using TF-IDF vectorization, the weighted Bag-of-Words representation, with unigrams and bigrams. Including bigrams allows the model to capture phrase-level patterns such as “free entry” or “claim now” that single words alone may miss.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features = 3000)
X = vectorizer.fit_transform(corpus)

X.shape

TF-IDF vectorization transforms the corpus into a sparse numerical feature matrix. The result contains up to 3,000 weighted unigram and bigram features.

##4.  Modeling and Evaluation

Target Variable Definition


In [ ]:
y = df['label_num']

Train-Test-Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,
                                                    random_state=42, stratify=y)
# Stratifcation preserves spam/ham imbalance ratio

### Evaluation Metrics

Because misclassifying emails has different consequences, the following metrics are the focus of this project:

- **Precision**: Of the emails flagged as spam, how many were actually spam?
  - High precision minimizes false positives (valid emails sent to spam).

- **Recall**: Of all spam emails, how many were correctly detected?
  - High recall minimizes false negatives (spam reaching the inbox).

- **False Positives (FP)**: Valid emails incorrectly flagged as spam.
- **False Negatives (FN)**: Spam emails incorrectly labeled as valid.


### Baseline Model (Majority Class)

In [ ]:
# Baseline model
import numpy as np
from sklearn.metrics import classification_report

# The baseline model predicts the majority class by assigning a value of 0 (ham)
# to every test instance using np.zeros_like(y_test)
y_pred = np.zeros_like(y_test)
print(classification_report(y_test, y_pred))


Warnings appear because the baseline model predicts only the majority class (ham), resulting in undefined precision for spam.

### Logistic Regression (default parameters)

In [ ]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

**Label Encoding:**  
- 0 = ham  
- 1 = spam

**Cnfusion Matrices:**
- Top-right cell: **False Positives** (ham incorrectly flagged as spam)
- Bottom-left cell: **False Negatives** (spam messages missed in prediction)



In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(7, 5))

im = ax.imshow(cm, cmap='Accent', interpolation='nearest')
fig.colorbar(im, ax=ax)

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['ham', 'spam'])
ax.set_yticklabels(['ham', 'spam'])

ax.set_xlabel('Predicted label')
ax.set_ylabel('Atual label')
ax.set_title('Confusion Matrix (Logistic Regression)')

for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha='center', va='center')

plt.show()


**Model Interpretation:**

Logistic Regression provides a strong linear baseline with high overall accuracy and good spam recall, while remaining interpretable and well-calibrated.

*Precision* reflects how often messages predicted as spam are truly spam, *recall* measures how many actual spam messages are identified, and the *F1-score* balances these tow metrics to account for class imbalance.

### Random Forest (default parameters)


In [ ]:
# This is the baseline Random Forest model.

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(7, 5))

im = ax.imshow(cm, cmap='Accent', interpolation='nearest')
fig.colorbar(im, ax=ax)

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['ham', 'spam'])
ax.set_yticklabels(['ham', 'spam'])

ax.set_xlabel('Predicted label')
ax.set_ylabel('Actual label')
ax.set_title('Confusion Matrix (Random Forest)')

for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha='center', va='center')

plt.show()


**Model Interpretation:**

The Random Forest improves overall performance by aggregating multiple decision trees, achieving high precision and strong recall for spam detection.

| Model                | False Positives | False Negatives |
|----------------------|-----------------|-----------------|
| Logistic Regression  | 0               | 38              |
| Random Forest        | 1               | 24              |

Compared to Logistic Regression, the Random Forest model detects 14 additional spam messages but incorrectly flags 1 non-spam messages.

### ROC and AUC Charts for Spam and Ham Classification

ROC and AUC charts are used to evaluate the performance among the 2 models in separating spam from ham, showing the trade-offs between true positive and false positive rates at various threshold settings.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

proba_lr = lr.predict_proba(X_test)[:,1]
proba_rf = rf.predict_proba(X_test)[:,1]

fpr_lr, tpr_lr, _ = roc_curve(y_test, proba_lr)
fpr_rf, tpr_rf, _ = roc_curve(y_test, proba_rf)

auc_lr = roc_auc_score(y_test, proba_lr)
auc_rf = roc_auc_score(y_test, proba_rf)

plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {auc_lr:.4f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.4f})')
plt.plot([0, 1], [0, 1], linestyle='--', label='Random')
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.show()

**ROC-AUC Interpretation:**

While the Random Forest model achieves higher recall and F1-score at the default classification threshold (as reflected in the confusion matrices), both models perform very well overall, as shown by their high ROC–AUC values. This suggests that much of the distinction between spam and ham can be explained by a simple linear model.





## Threshold Tuning (Decision Tradeoffs)

By default, classification models use a probability threshold of 0.5 to assign class labels.
However, this threshold may not be optimal for all real-world use cases.

In spam detection, lowering the threshold can increase spam recall at the cost of more false positives,
while raising the threshold prioritizes precision by reducing the chance of blocking legitimate emails.

The interactive slider below demonstrates how adjusting the decision threshold impacts model performance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
y_proba = lr.predict_proba(X_test)[:, 1]

In [ ]:
def update_threshold(threshold):
    clear_output(wait=True)

    # Apply threshold
    y_pred_thresh = (y_proba >= threshold).astype(int)

    # Metrics
    precision = precision_score(y_test, y_pred_thresh)
    recall = recall_score(y_test, y_pred_thresh)
    f1 = f1_score(y_test, y_pred_thresh)

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_thresh)

    print(f"Threshold: {threshold:.2f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall:    {recall:.3f}")
    print(f"F1-score:  {f1:.3f}\n")

    # plt.figure(figsize=(6, 4))
    plt.imshow(cm, cmap='Accent', interpolation='nearest')
    plt.colorbar()
    plt.xticks([0, 1], ['ham', 'spam'])
    plt.yticks([0, 1], ['ham', 'spam'])

    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('Actual Label')

    for i in range(2):
        for j in range(2):
            plt.text(j, i, cm[i, j], ha='center', va='center')

    plt.tight_layout()
    plt.show()


In [ ]:
threshold_slider = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=0.9,
    step=0.05,
    description='Threshold:',
    continuous_update=False
)

widgets.interact(update_threshold, threshold=threshold_slider)


**Observation:**  
Lowering the threshold increases spam recall but introduces more false positives,
while higher thresholds improve precision at the risk of missing spam messages.

This highlights the importance of selecting a threshold that aligns with business priorities
rather than relying solely on the default value.


##5. Exploratory Optional Extensions

### Bag of Words (Baseline - Exploratory)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer(max_features=3000, stop_words='english')

X_bow = bow.fit_transform(corpus)
Xb_train, Xb_test, yb_train, yb_test = train_test_split(X_bow, y, test_size=.25,
                                                    random_state=42, stratify=y)
lr_bow = LogisticRegression()
# lr_bow = LogisticRegression(max_iter=1000)
lr_bow.fit(Xb_train, yb_train)

yb_pred = lr_bow.predict(Xb_test)
print(classification_report(yb_test, yb_pred))

As an exploratory comparison, a Bag-of-Words (raw word counts) representation is evaluated using Logistic Regression. Although Bag-of-Words achieves slightly higher spam recall on this dataset, its dependence on raw term frequency makes it more sensitive to dataset-specific words. TF-IDF is selected as the preferred representation because it provides greater robustness to various language patterns and better generalization on the dataset.

## Transformer-Based Model (BERT / Hugging Face – Exploratory)

In [ ]:
from transformers import pipeline

bert_classifier = pipeline(
    'text-classification',
    model = 'distilbert-base-uncased-finetuned-sst-2-english',
    tokenizer = 'distilbert-base-uncased-finetuned-sst-2-english'
)

sample_text = df['message'].sample(10, random_state=42).tolist()
bert_predictions = bert_classifier(sample_text)
list(zip(sample_text, bert_predictions)
)

**Transformer-Based Model (Exploratory):**

A pretrained DistilBERT model from Hugging Face is explored using the transformers pipeline to explore transformer-based NLP models. The model is fine-tuned for sentiment analysis rather than spam classification, so results are not directly comparable to supervised models trained on the dataset. Due to this mismatch and higher computational cost, classical TF-IDF–based models remain the preferred solution for this task.

##6. Final Summary
This project evaluates multiple supervised learning approaches for spam classification using TF-IDF text features with an imbalance class distribution. A majority-class baseline demonstrates that accuracy alone is insufficient, prompting the need for precision, recall, F1-score, and ROC–AUC for evaluation.

Logistic Regression and Random Forest models both achieve strong performance. Logistic Regression provides a stable and interpretable linear baseline with reliable probability estimates, while the default Random Forest model achieves higher precision and strong recall by aggregating multiple decision trees. Excessive regularization of the Random Forest leads to underfitting and reduced spam recall, illustrating the importance of balancing model complexity with dataset characteristics.

Logistic Regression serves as a strong and interpretable baseline. However, final model selection depends on the relative cost of false positives versus false negatives, which is addressed in the Recommendation section.



##7. Recommendation
Alternative feature representations, including bigrams and Bag-of-Words, produce results comparable to unigram TF-IDF, indicating stable model behavior and the keyword-driven nature of SMS spam. A pretrained BERT-based model from Hugging Face is explored to gain exposure to modern NLP techniques, but does not provide a clear advantage for this short-text classification task due to higher computational cost and the lack of task-specific fine-tuning.

**For spam filtering applications where false positives are more costly than false negatives, the default Random Forest model is preferred due to its high precision and robust performance.** Logistic Regression remains a strong alternative when interpretability and simplicity are prioritized.